In [3]:
import os
import sys
import numpy as np
import essentia.standard as ess
from scipy.fftpack import fft
import matplotlib.pyplot as plt
import csv
import pydub
import pandas as pd
from pathlib import Path
from pydub import AudioSegment
from pydub.utils import make_chunks


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import IPython.display as ipd

In [4]:
#Reading a sample wave file
soundsDIR = '../gnaimeh/fulltracks/'
fs = 44100

x = ess.MonoLoader(filename = soundsDIR + 'stilvortalentarchive/2020/01-2020/09/Brigado Crew - Equilibrium (Stil Vor Talent)/02-Brigado Crew - Ritual (Original Mix)-electrobuzz.net.mp3', sampleRate = fs)()
#Normalize maximum amplitude to 1
x = x / np.max(np.abs(x))
#Setting the parameters
windowSize = 1024
hopSize = 512
startIndexes = np.arange(0, x.size - windowSize, hopSize, dtype = int)#frame/window start indexes
numWindows = startIndexes.size

In [5]:
# Raw-data preprocess analysis parameters
fs = 44100
windowSize = 4096 * 4
hopSize = 4096 * 2
NRG_threshold_ratio = 0.01 #threshold expressed as ratio with respect to the maximum value
#Let's put in a container to be able to use as a single argument in function calls
params = {"fs":fs, "windowSize":windowSize, "hopSize":hopSize, "NRG_threshold_ratio": NRG_threshold_ratio}

In [6]:
count = 0
count_songs = 0
labels = []
labels_dict = dict()


for root, dirs, files in os.walk ('../gnaimeh/fulltracks/'):
    count += 1
    if count==1:
        labels = [label for label in dirs if not label.startswith('.')]
    else:
        break
        
for l in labels:
    count_songs = 0
    for root, dirs, files in os.walk('../gnaimeh/fulltracks/'+l):
        for file in files:
            if  file.endswith('.mp3'):
                count_songs += 1
    
                
    print(l, " songs ", count_songs)       
    if "name" in labels_dict:
        labels_dict["name"].append(l)
        labels_dict['number_songs'].append(count_songs)
    else:
        labels_dict["name"] = [l]
        labels_dict["number_songs"] = [count_songs]
        

suaraarchive  songs  571
mordarchive  songs  831
chillhoparchive  songs  562
defectedarchive  songs  1560
stilvortalentarchive  songs  580
ninjatunearchive  songs  610
bar25archive  songs  422
littlehelpersarchive  songs  720
kompaktarchive  songs  329


In [7]:
def NRG_time(x, filename, params):
    '''Function to define split boundaries based on a fixed energy threshold
    '''
    NRG = [];
    #Main windowing and feature extraction loop
    for frame in ess.FrameGenerator(x, frameSize = windowSize, hopSize = hopSize, startFromZero = True):
        NRG.append(ess.Energy()(frame))
    NRG = np.array(NRG)
    NRG = NRG / np.max(NRG)
    tNRG = (np.argmax(NRG)*hopSize)#/fs
    #t_total= (x.size)/fs
    #pos_NRG = (tNRG/t_total)*100
    
    return (tNRG)

In [8]:
chunks_path = '../gnaimeh/NRG_chunks_120s'
countlabel = 0
count = 0
nfiles = 610
fs = 44100
for l in labels_dict['name']:
        # for each label
        print(l)
        count = 0
        for root, dirs, files in os.walk ('../gnaimeh/fulltracks/'+l):
            # iterate files at label directory
            # extract features from at most nfiles which is the minimal
            # number of files in all labels
            if count >= nfiles:
                count = 0
                break
            for file in files:
                if not file.startswith('.') and file.endswith('.mp3'):
                    # if file is a song
                    # extract features and write them to csv
                    # increase count to stop at nfiles
                    if count < nfiles:
                        count += 1
                        # display progress every 20 files
                        if count % 1 == 0:
                            print(count, "files processed, current file: ",file)
                            filename = root + '/' + file
                            
                            try:
                                x = ess.MonoLoader(filename= filename, sampleRate = fs)()
                                t=NRG_time(x, root +'/' + file, params)
                            except:
                                continue
                            print(t)
                            
                            if t>fs*60 and t<(len(x)-fs*60):
                                e_start = t-int(fs*60)
                                e_finish = t+int(fs*60)
                                chunk = x[e_start:e_finish]
                                chunking_dir = os.path.join(chunks_path, l)  #filename.split("/")[-3]
                                if not os.path.exists(chunking_dir):
                                    os.makedirs(chunking_dir)
                                chunk_dir = os.path.join(chunking_dir, filename.split(".")[0])
                                chunk_path = os.path.join(chunk_dir, file.split('.mp3')[0]+ '.wav')
                                #chunk.export(chunk_path, format='mp3') 
                                ess.MonoWriter(filename=chunk_path, format= 'wav', sampleRate=fs)(chunk)
                                #np.save(chunk_path, chunk)
                                print("Exporting", chunk_path)
                            else:
                                print('not enough time', filename)
                                continue
                            
            
                        else:
                        # we extracted nfiles start a new label
                            print(l)
                            print(count)
                            print("Another label")
                            break

suaraarchive
1 files processed, current file:  03-ulm_west_deep-poem_(drum_version)-electrobuzz.net.mp3
1318912
not enough time ../gnaimeh/fulltracks/suaraarchive/2012/05-2012/19/Ulm_West_Deep-Here_We_Are_EP_SUARA051/03-ulm_west_deep-poem_(drum_version)-electrobuzz.net.mp3
2 files processed, current file:  13-heartik-wheres_the_beat-electrobuzz.net.mp3
1957888
not enough time ../gnaimeh/fulltracks/suaraarchive/2012/05-2012/09/VA-Suara_Showroom_Retrospected_SCOM007/13-heartik-wheres_the_beat-electrobuzz.net.mp3
3 files processed, current file:  02-pleasurekraft_and_belocca-murdered_out_(samuel_dan_and_darlyn_vlys_remix)-electrobuzz.net.mp3
802816
not enough time ../gnaimeh/fulltracks/suaraarchive/2012/05-2012/22/Belocca_and_Pleasurekraft-Murdered_Out_EP_SUARA052/02-pleasurekraft_and_belocca-murdered_out_(samuel_dan_and_darlyn_vlys_remix)-electrobuzz.net.mp3
4 files processed, current file:  02-AFFKT-Aoub-electrobuzz.net.mp3
368640
not enough time ../gnaimeh/fulltracks/suaraarchive/2012/

Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Ramiro Lopez - Real Party..wav
39 files processed, current file:  Ramiro Lopez - Filgud  (Coyu 'Feelraw' Remix).mp3
4390912
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Ramiro Lopez - Filgud  (Coyu 'Feelraw' Remix).wav
40 files processed, current file:  Edu Imbernon & Triumph - Neim (Milton Jackson Remix).mp3
5013504
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Edu Imbernon & Triumph - Neim (Milton Jackson Remix).wav
41 files processed, current file:  Edu Imbernon & Triumph - Neim.mp3
9641984
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Edu Imbernon & Triumph - Neim.wav
42 files processed, current file:  04-sergio_fernandez_and_david_lara-deepha_(original_mix)-electrobuzz.net.mp3
13287424
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/04-sergio_fernandez_and_david_lara-deepha_(original_mix)-electrobuzz.net.wav
43 files processed, current file:  03-jon_rundell-tip_top_(original_mix)-electrobuzz.net.mp3
13287424
Exportin

21831680
not enough time ../gnaimeh/fulltracks/suaraarchive/2016/05-2016/05/Nick & Danny Chatelain - Acid EP [Suara - SUARA222]/Nick & Danny Chatelain - Acid (Original Mix) [Suara].mp3
78 files processed, current file:  Nick & Danny Chatelain - Acid (Coyu Tool Mix) [Suara].mp3
7503872
not enough time ../gnaimeh/fulltracks/suaraarchive/2016/05-2016/05/Nick & Danny Chatelain - Acid EP [Suara - SUARA222]/Nick & Danny Chatelain - Acid (Coyu Tool Mix) [Suara].mp3
79 files processed, current file:  Shiba San - Got Drunk (Original Mix) [Suara].mp3
11829248
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Shiba San - Got Drunk (Original Mix) [Suara].wav
80 files processed, current file:  Shiba San - Hard Day (German Brigante Remix) [Suara].mp3
12673024
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Shiba San - Hard Day (German Brigante Remix) [Suara].wav
81 files processed, current file:  Adrian Hour - Avalon (Original Mix) [Suara].mp3
4677632
Exporting ../gnaimeh/NRG_chunks_120s/suaraarch

1761280
not enough time ../gnaimeh/fulltracks/suaraarchive/2016/12-2016/06/Mark Fanciulli, Rob Cockerton - Thought I'd Rearrange Your Mind EP [Suara]/Rob Cockerton, Mark Fanciulli - Voices In My Head (Original Mix) [Suara].mp3
116 files processed, current file:  Mark Fanciulli, Rob Cockerton - Patterns (Original Mix) [Suara].mp3
16990208
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Mark Fanciulli, Rob Cockerton - Patterns (Original Mix) [Suara].wav
117 files processed, current file:  Nasser Baker - Kronos (Original Mix) [Suara].mp3
13246464
not enough time ../gnaimeh/fulltracks/suaraarchive/2016/12-2016/28(FTP_ONLY)/Nasser Baker - Gramercy EP [Suara]/Nasser Baker - Kronos (Original Mix) [Suara].mp3
118 files processed, current file:  Pretty Pink-Gunfire feat. Janine Villforth-electrobuzz.mp3
13524992
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Pretty Pink-Gunfire feat. Janine Villforth-electrobuzz.wav
119 files processed, current file:  Pretty Pink-What Is Love feat. Tears &

6864896
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Audio Junkies-Vividness feat. Haptic (Maxxi Soundsystem Remix)-electrobuzz.wav
152 files processed, current file:  Coyu Sezer Uysal-Cygnus (Original Mix)-electrobuzz.mp3
17170432
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Coyu Sezer Uysal-Cygnus (Original Mix)-electrobuzz.wav
153 files processed, current file:  Leftwing & Kody-8 Track-electrobuzz.mp3
15400960
not enough time ../gnaimeh/fulltracks/suaraarchive/2015/01-2015/06/Leftwing & Kody - The Kick Drum EP [SUARA161]/Leftwing & Kody-8 Track-electrobuzz.mp3
154 files processed, current file:  Fat Sushi - The Groove [Suara].mp3
11476992
Exporting ../gnaimeh/NRG_chunks_120s/suaraarchive/Fat Sushi - The Groove [Suara].wav
155 files processed, current file:  DJ Boris-One Nite (Original Mix)-electrobuzz.mp3
2023424
not enough time ../gnaimeh/fulltracks/suaraarchive/2015/01-2015/17/DJ Boris - Techno Tools Vol. 10 [SUARA162]/DJ Boris-One Nite (Original Mix)-electrobuzz.mp3
156

...

1220608
not enough time ../gnaimeh/fulltracks/kompaktarchive/2018/03-2018/08-09/Kenneth James Gibson - In The Fields Of Nothing [Kompakt]/Kenneth James Gibson - Further From Home (Original Mix) [Kompakt]-electrobuzz.net.mp3
172 files processed, current file:  Kenneth James Gibson - Hearing Unheard (Original Mix) [Kompakt]-electrobuzz.net.mp3
12689408
not enough time ../gnaimeh/fulltracks/kompaktarchive/2018/03-2018/08-09/Kenneth James Gibson - In The Fields Of Nothing [Kompakt]/Kenneth James Gibson - Hearing Unheard (Original Mix) [Kompakt]-electrobuzz.net.mp3
173 files processed, current file:  Kenneth James Gibson - To Love A Rotting Piano (Original Mix) [Kompakt]-electrobuzz.net.mp3
1531904
not enough time ../gnaimeh/fulltracks/kompaktarchive/2018/03-2018/08-09/Kenneth James Gibson - In The Fields Of Nothing [Kompakt]/Kenneth James Gibson - To Love A Rotting Piano (Original Mix) [Kompakt]-electrobuzz.net.mp3
174 files processed, current file:  Anii - Korzenie (Original Mix)_Kompakt.

10067968
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/Blondish-Inward Visions feat. Beyou.wav
212 files processed, current file:  Blondish-No Place Like Gnome.mp3
20455424
not enough time ../gnaimeh/fulltracks/kompaktarchive/2013/07-2013/22/Blondish - Inward Visions [KOMPAKT268]/Blondish-No Place Like Gnome.mp3
213 files processed, current file:  01-Saschienne-Grand Cru (Extended Version)-electrobuzz.net.mp3
15228928
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/01-Saschienne-Grand Cru (Extended Version)-electrobuzz.net.wav
214 files processed, current file:  04-CoMa-My Orbit (Single Edit)-electrobuzz.net.mp3
11829248
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/04-CoMa-My Orbit (Single Edit)-electrobuzz.net.wav
215 files processed, current file:  03-CoMa-My Orbit (Dauwd Mix)-electrobuzz.net.mp3
3383296
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/03-CoMa-My Orbit (Dauwd Mix)-electrobuzz.net.wav
216 files processed, current file:  01-CoMa-My Orbit (Clouds Mix

15532032
not enough time ../gnaimeh/fulltracks/kompaktarchive/2014/05-2014/20/VA - Momento Drive [Kompakt]/Christian S.-Jagos [Kompakt]-electrobuzz.mp3
250 files processed, current file:  Various Artists-Continuous Album Mix [Kompakt]-electrobuzz.mp3
28598272
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/Various Artists-Continuous Album Mix [Kompakt]-electrobuzz.wav
251 files processed, current file:  Wolfgang Voigt-Du Musst Nichts Sagen (Fanfaren Mix) [Kompakt]-electrobuzz.mp3
15892480
not enough time ../gnaimeh/fulltracks/kompaktarchive/2014/05-2014/20/VA - Momento Drive [Kompakt]/Wolfgang Voigt-Du Musst Nichts Sagen (Fanfaren Mix) [Kompakt]-electrobuzz.mp3
252 files processed, current file:  Sebastien Bouchet-Tempestuous [Kompakt]-electrobuzz.mp3
5586944
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/Sebastien Bouchet-Tempestuous [Kompakt]-electrobuzz.wav
253 files processed, current file:  Rebolledo-Windsurf  Sunburn and Dollar [Kompakt]-electrobuzz.mp3
10477568
Exportin

11141120
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/01-Rebolledo, The Novotones - Mountain Eagle (The Black Frame Desert Mix)-electrobuzz.net.wav
287 files processed, current file:  01-Weval - Heaven, Listen (Original Mix)-electrobuzz.net.mp3
5496832
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/01-Weval - Heaven, Listen (Original Mix)-electrobuzz.net.wav
288 files processed, current file:  01-Blackrachas - Rotary (Original Mix)-electrobuzz.net.mp3
15974400
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/01-Blackrachas - Rotary (Original Mix)-electrobuzz.net.wav
289 files processed, current file:  02-Anii - Aniitime (Original Mix)-electrobuzz.net.mp3
13942784
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/02-Anii - Aniitime (Original Mix)-electrobuzz.net.wav
290 files processed, current file:  01-Anii - Ride The Tiger (Original Mix)-electrobuzz.net.mp3
4333568
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/01-Anii - Ride The Tiger (Original Mix)-electrobuzz

11616256
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/23-Raxon - Dark Light (Original Mix)-electrobuzz.net.wav
323 files processed, current file:  25-Jonathan Kaspar - Renard (Original Mix)-electrobuzz.net.mp3
10403840
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/25-Jonathan Kaspar - Renard (Original Mix)-electrobuzz.net.wav
324 files processed, current file:  08-Reinhard Voigt - Der Mann, der nie nach Deutz kam (Original Mix)-electrobuzz.net.mp3
12935168
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/08-Reinhard Voigt - Der Mann, der nie nach Deutz kam (Original Mix)-electrobuzz.net.wav
325 files processed, current file:  16-DJ Balduin - E.W.B.A. (Original Mix)-electrobuzz.net.mp3
15065088
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/16-DJ Balduin - E.W.B.A. (Original Mix)-electrobuzz.net.wav
326 files processed, current file:  17-ANNA - Remembrance (Main Mix)-electrobuzz.net.mp3
11558912
Exporting ../gnaimeh/NRG_chunks_120s/kompaktarchive/17-ANNA - Remembran

NUmpies

In [ ]:
chunks_path = '../gnaimeh/NRG_chunks'
countlabel = 0
count = 0
nfiles = 329
fs = 44100
for l in labels_dict['name']:
        # for each label
        print(l)
        count = 0
        for root, dirs, files in os.walk ('../gnaimeh/fulltracks/'+l):
            # iterate files at label directory
            # extract features from at most nfiles which is the minimal
            # number of files in all labels
            if count >= nfiles:
                count = 0
                break
            for file in files:
                if not file.startswith('.') and file.endswith('.mp3'):
                    # if file is a song
                    # extract features and write them to csv
                    # increase count to stop at nfiles
                    if count < nfiles:
                        count += 1
                        # display progress every 20 files
                        if count % 1 == 0:
                            print(count, "files processed, current file: ",file)
                            filename = root + '/' + file
                            
                            try:
                                x = ess.MonoLoader(filename= filename, sampleRate = fs)()
                                t=NRG_time(x, root +'/' + file, params)
                            except:
                                continue
                            print(t)
                            
                            if t>fs/2 and t<(len(x)-fs/2):
                                e_start = t-int(fs/2)
                                e_finish = t+int(fs/2)
                                chunk = x[e_start:e_finish]
                                chunking_dir = os.path.join(chunks_path, l)  #filename.split("/")[-3]
                                if not os.path.exists(chunking_dir):
                                    os.makedirs(chunking_dir)
                                chunk_dir = os.path.join(chunking_dir, filename.split(".")[0])
                                chunk_path = os.path.join(chunk_dir, file.split('.mp3')[0]+'.npy')
                                #chunk.export(chunk_path, format='mp3') 
                                
                                np.save(chunk_path, chunk)
                                print("Exporting", chunk_path)
                            else:
                                print('not enough time', filename)
                                continue
                            
            
                        else:
                        # we extracted nfiles start a new label
                            print(l)
                            print(count)
                            print("Another label")
                            break